# Capstone Project for IBM Data Science Certificate

## Segmenting and Clustering Neighborhoods in Toronto
## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)

## Introduction: Business Problem <a name="introduction"></a>

In this project we will try to find an optimal location for a restaurant. Specifically, this report will be targeted to stakeholders interested in opening an **Asian restaurant** in **Toronto**, Canada.

Since there are lots of restaurants in Toronto we will try to detect **locations that are not already crowded with restaurants**. We are also particularly interested in **areas with limited Asian restaurants in vicinity**. We would also prefer locations **where there are more venues in total and further from the city center**, assuming that first two conditions are met.

We will also cluster the neighborhoods based on the venues to better understand the business environment of each neighborhood. 

We will use our data science powers to generate a few most promising neighborhoods based on this criteria. Advantages of each area will then be clearly expressed so that best possible final location can be chosen by stakeholders.

## Data <a name="data"></a>

Based on definition of our problem, factors that will influence our decission are:
* number of existing restaurants in the neighborhood (any type of restaurant)
* number of and distance to Italian restaurants in the neighborhood, if any
* Average rating of restaurants
* The venue cluster of the neighborhood and the main characteristic of each neighborhood


Following data sources will be needed to extract/generate the required information:
* number of restaurants and their type and location in every neighborhood will be obtained using **Foursquare API**

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

!conda install -c conda-forge beautifulsoup4 --yes
from bs4 import BeautifulSoup
!conda install -c conda-forge lxml --yes
import lxml

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          92 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-1.21.0-py_0



geopy-1.21.0         | 58 KB     | ##################################### | 100% 
geographiclib-1.50   | 34 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Sol

In [2]:

res = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(res.content, features='html')
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))
print(df[0].shape)
tor_data = df[0]
tor_data = tor_data[tor_data['Borough']!='Not assigned']
print(tor_data.shape)

(180, 3)
(103, 3)


In [3]:
tor_data.reset_index(inplace = True)
tor_data.drop(['index'], axis = 1,inplace = True)


/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [4]:
tor_data.shape

(103, 3)

In [5]:
tor_data.head()


,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


## Get the Latitude and Longitude Data of Each Neighborhood

In [6]:
geo_data = pd.read_csv('Geospatial_Coordinates.csv')
geo_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [7]:
tor_data = tor_data.merge(geo_data, how = 'left',left_on = 'Postal code', right_on = 'Postal Code')
tor_data.drop(['Postal Code'], axis = 1, inplace = True)
tor_data.head()

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636
3,M6A,North York,Lawrence Manor / Lawrence Heights,43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494


## Define Foursquare Info

In [8]:
CLIENT_ID = 'Y4DKOYK4CQ2OYJNNVPJA5YJLQUOGBA0W1SVMFQBED2R5ZBK3' # your Foursquare ID
CLIENT_SECRET = 'LHYPOEWCPURSACTYWPUB02HTUURYGBXLMURMD42ZTIOLQYSR' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: Y4DKOYK4CQ2OYJNNVPJA5YJLQUOGBA0W1SVMFQBED2R5ZBK3
CLIENT_SECRET:LHYPOEWCPURSACTYWPUB02HTUURYGBXLMURMD42ZTIOLQYSR


Import necessary packages

Try get the Top 50 Venues in Postal Code M5A with a Radius of 500

In [9]:
dt_tor_lat = tor_data.loc[tor_data['Postal code']=='M5A',:]['Latitude']
dt_tor_long = tor_data.loc[tor_data['Postal code']=='M5A','Longitude']
dt_tor_neighborhood_name = tor_data.loc[tor_data['Postal code']=='M5A','Neighborhood']




In [10]:
radius = 500
LIMIT = 50

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, 43.65426, -79.360636, VERSION, radius, LIMIT)

In [11]:
results_m5a = requests.get(url).json()

In [12]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [14]:
from pandas.io.json import json_normalize
venues_m5a = json_normalize(results_m5a['response']['venues'])

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  


In [15]:
filtered_columns = ['name', 'categories', 'location.lat', 'location.lng']
nearby_venues_m5a = venues_m5a.loc[:, filtered_columns]

# filter the category for each row
nearby_venues_m5a['categories'] = nearby_venues_m5a.apply(get_category_type, axis=1)

# clean columns
nearby_venues_m5a.columns = [col.split(".")[-1] for col in nearby_venues_m5a.columns]

nearby_venues_m5a.head()

,name,categories,lat,lng
0,Oldtown Bodega,Café,43.653966,-79.360752
1,Sackville Playground,Park,43.654656,-79.359871
2,Tandem Coffee,Coffee Shop,43.653559,-79.361809
3,Morning Glory Cafe,Breakfast Spot,43.653947,-79.361149
4,Body Blitz Spa East,Spa,43.654735,-79.359874


In [16]:
unique_cat = nearby_venues_m5a['categories'].nunique()

In [17]:
print('There are {} unique categories of veunes around postal code M5A.'.format(unique_cat))

There are 38 unique categories of veunes around postal code M5A.


## Analysis <a name="analysis"></a>

Let's perform some basic explanatory data analysis and derive some additional info from our raw data. First let's count the **number of restaurants in every neighborhood**:

Get all venues information of all neighborhoods

In [18]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name'],
            v['venue']['categories'][0]['id']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category',
                  'Category ID']
    
    return(nearby_venues)

In [19]:
CLIENT_ID = 'Y4DKOYK4CQ2OYJNNVPJA5YJLQUOGBA0W1SVMFQBED2R5ZBK3' # your Foursquare ID
CLIENT_SECRET = 'LHYPOEWCPURSACTYWPUB02HTUURYGBXLMURMD42ZTIOLQYSR' # your Foursquare Secret
VERSION = '20180605'
LIMIT = 100
neigh_names = tor_data['Neighborhood'].tolist()
lat_list = tor_data['Latitude'].tolist()
long_list = tor_data['Longitude'].tolist()

In [20]:
tor_venues = getNearbyVenues(names=neigh_names,
                                   latitudes=lat_list,
                                   longitudes=long_list
                                  )

In [21]:
# Add two empty columns
tor_venues['Is Restaurant'] = np.nan
tor_venues['Asian Restaurant'] = np.nan
tor_venues.shape

(4945, 10)

In [22]:
tor_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Category ID,Is Restaurant,Asian Restaurant
0,Parkwoods,43.753259,-79.329656,Allwyn's Bakery,43.759840,-79.324719,Caribbean Restaurant,4bf58dd8d48988d144941735,NaN,NaN
1,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park,4bf58dd8d48988d163941735,NaN,NaN
2,Parkwoods,43.753259,-79.329656,Tim Hortons,43.760668,-79.326368,Café,4bf58dd8d48988d16d941735,NaN,NaN
3,Parkwoods,43.753259,-79.329656,A&W,43.760643,-79.326865,Fast Food Restaurant,4bf58dd8d48988d16e941735,NaN,NaN
4,Parkwoods,43.753259,-79.329656,Bruno's valu-mart,43.746143,-79.324630,Grocery Store,4bf58dd8d48988d118951735,NaN,NaN


In [23]:
# Category IDs corresponding to Italian restaurants were taken from Foursquare web site (https://developer.foursquare.com/docs/resources/categories):
food_catid = '4d4b7105d754a06374d81259'
asian_rest_catid = ['52af3a7c3cf9994f4e043bed','58daa1558bbb0b01f18ec1d3','4bf58dd8d48988d1f5931735','52af3a9f3cf9994f4e043bef','52af3aaa3cf9994f4e043bf0','52af3ac83cf9994f4e043bf3','52af3afc3cf9994f4e043bf8','52af3b463cf9994f4e043bfe',
                    '52af3b593cf9994f4e043c00','52af3b773cf9994f4e043c03','52af3b813cf9994f4e043c04','52af3b913cf9994f4e043c06','4eb1bd1c3b7b55596b4a748f','52e81612bcbc57f1066b79fb','52af0bd33cf9994f4e043bdd',
                   '4deefc054765f83613cdba6f','4bf58dd8d48988d111941735','55a59bace4b013909087cb24','55a59bace4b013909087cb15','55a59bace4b013909087cb27','4bf58dd8d48988d1d2941735','55a59bace4b013909087cb2a','4bf58dd8d48988d113941735',
                   '4bf58dd8d48988d156941735','4eb1d5724b900d56c88a45fe','4bf58dd8d48988d1d1941735','56aa371be4b08b9a8d57350e','4bf58dd8d48988d149941735','4bf58dd8d48988d14a941735']


In [24]:
# See if the restaurant belongs to a specific category
def is_restaurant(df):
    restaurant_words = ['Restaurant', 'Diner', 'Taverna', 'Steakhouse']
    
    if 'Fast Food' in df['Venue Category']:
        return False
    else:
        for r in restaurant_words:
            if r in df['Venue Category']:
                return True
            else:
                return False

def asian_restaurant(df):
    asian_rest_catid = ['52af3a7c3cf9994f4e043bed','58daa1558bbb0b01f18ec1d3','4bf58dd8d48988d1f5931735','52af3a9f3cf9994f4e043bef','52af3aaa3cf9994f4e043bf0','52af3ac83cf9994f4e043bf3','52af3afc3cf9994f4e043bf8','52af3b463cf9994f4e043bfe',
                    '52af3b593cf9994f4e043c00','52af3b773cf9994f4e043c03','52af3b813cf9994f4e043c04','52af3b913cf9994f4e043c06','4eb1bd1c3b7b55596b4a748f','52e81612bcbc57f1066b79fb','52af0bd33cf9994f4e043bdd',
                   '4deefc054765f83613cdba6f','4bf58dd8d48988d111941735','55a59bace4b013909087cb24','55a59bace4b013909087cb15','55a59bace4b013909087cb27','4bf58dd8d48988d1d2941735','55a59bace4b013909087cb2a','4bf58dd8d48988d113941735',
                   '4bf58dd8d48988d156941735','4eb1d5724b900d56c88a45fe','4bf58dd8d48988d1d1941735','56aa371be4b08b9a8d57350e','4bf58dd8d48988d149941735','4bf58dd8d48988d14a941735']
    
    if df['Category ID'] in asian_rest_catid:
        return True
    else:
        return False


In [25]:
tor_venues['Is Restaurant'] = tor_venues.apply(lambda row: is_restaurant(row),axis = 1)
tor_venues['Asian Restaurant'] = tor_venues.apply(lambda row: asian_restaurant(row),axis = 1)


In [26]:
#Get the total number of restaurants and asian restaurants in toronto
Total_rest_num = tor_venues['Is Restaurant'].value_counts()[1]
Total_asian_rest = tor_venues['Asian Restaurant'].value_counts()[1]

## Check the restaurant number and proportion to general venues

In [27]:
rest_percent = tor_venues.groupby('Neighborhood')[['Is Restaurant','Asian Restaurant']].mean()
rest_percent.reset_index(inplace = True)

In [28]:
venue_count = tor_venues.groupby('Neighborhood')['Asian Restaurant'].count()
venue_count = venue_count.reset_index()
rest_percent['Venue Count'] = venue_count['Asian Restaurant']
rest_percent['Asian Restaurant Percentage'] = rest_percent['Asian Restaurant']/rest_percent['Is Restaurant']

In [29]:
rest_percent

,Neighborhood,Is Restaurant,Asian Restaurant,Venue Count,Asian Restaurant Percentage
0,Agincourt,0.450980,0.156863,51,0.347826
1,Alderwood / Long Branch,0.041667,0.000000,24,0.000000
2,Bathurst Manor / Wilson Heights / Downsview North,0.166667,0.033333,30,0.200000
3,Bayview Village,0.307692,0.153846,13,0.500000
4,Bedford Park / Lawrence Manor East,0.268293,0.048780,41,0.181818
5,Berczy Park,0.200000,0.060000,100,0.300000
6,Birch Cliff / Cliffside West,0.133333,0.066667,15,0.500000
7,Brockton / Parkdale Village / Exhibition Place,0.250000,0.010000,100,0.040000
8,Business reply mail Processing CentrE,0.166667,0.062500,48,0.375000
9,CN Tower / King and Spadina / Railway Lands / ...,0.000000,0.000000,15,NaN


In [30]:
avg_rest_percent = rest_percent['Is Restaurant'].mean()
avg_asian_percent = rest_percent['Asian Restaurant'].mean()
med_venue_count = rest_percent['Venue Count'].median()
asian_avg = rest_percent['Asian Restaurant Percentage'].mean()
avg_rest_percent, avg_asian_percent, med_venue_count, asian_avg

(0.19765704134195589, 0.05336631675121712, 41.0, 0.2582892121776855)

In [31]:
rest_percent[(rest_percent['Is Restaurant']<avg_rest_percent) & (rest_percent['Asian Restaurant Percentage']<asian_avg)].shape

(18, 5)

In [33]:
potential_neigh = rest_percent[(rest_percent['Is Restaurant']<avg_rest_percent) & (rest_percent['Asian Restaurant']<avg_asian_percent) & (rest_percent['Venue Count']>= med_venue_count)]
print('From the table below we can see that are the primary neighborhoods that we can choose from.')
neigh_list = potential_neigh['Neighborhood'].tolist()
neigh_list

From the table below we can see that are the primary neighborhoods that we can choose from.


['Fairview / Henry Farm / Oriole',
 'Leaside',
 'Regent Park / Harbourfront',
 'Runnymede / The Junction North']

## Analyze Neighborhood by Clustering

In [34]:
tor_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Category ID,Is Restaurant,Asian Restaurant
0,Parkwoods,43.753259,-79.329656,Allwyn's Bakery,43.759840,-79.324719,Caribbean Restaurant,4bf58dd8d48988d144941735,True,False
1,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park,4bf58dd8d48988d163941735,False,False
2,Parkwoods,43.753259,-79.329656,Tim Hortons,43.760668,-79.326368,Café,4bf58dd8d48988d16d941735,False,False
3,Parkwoods,43.753259,-79.329656,A&W,43.760643,-79.326865,Fast Food Restaurant,4bf58dd8d48988d16e941735,False,False
4,Parkwoods,43.753259,-79.329656,Bruno's valu-mart,43.746143,-79.324630,Grocery Store,4bf58dd8d48988d118951735,False,False


Modify the Venue Category to Restaurant in General

In [39]:
def switch_category(row):
    if row['Is Restaurant'] == True:
        return 'Restaurant'
    else:
        return row['Venue Category']


In [41]:
tor_venues_trans = tor_venues.loc[:,['Neighborhood','Venue Category','Is Restaurant']]
tor_venues_trans['Venue Category'] = tor_venues_trans.apply(lambda row: switch_category(row), axis = 1)
tor_venues_trans.drop(columns = 'Is Restaurant', inplace = True)
tor_venues_trans.head()

,Neighborhood,Venue Category
0,Parkwoods,Restaurant
1,Parkwoods,Park
2,Parkwoods,Café
3,Parkwoods,Fast Food Restaurant
4,Parkwoods,Grocery Store


In [50]:
# One hot encoding
tor_onehot = pd.get_dummies(tor_venues_trans[['Venue Category']], prefix="", prefix_sep="")

#add neighborhood column back to dataframe
tor_onehot['Neighborhood'] = tor_venues_trans['Neighborhood'] 


tor_onehot.shape

(4945, 270)

In [51]:
tor_grouped = tor_onehot.groupby('Neighborhood').mean().reset_index()


,Neighborhood,Accessories Store,Airport,Airport Lounge,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Arcade,Art Gallery,Arts & Crafts Store,Athletics & Sports,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Bike Shop,Bistro,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Camera Store,Candy Store,Castle,Cemetery,Cheese Shop,Chiropractor,Chocolate Shop,Church,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Lab,College Quad,College Rec Center,College Stadium,College Theater,Comedy Club,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Dentist's Office,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Donut Shop,Drugstore,Electronics Store,Event Space,Farm,Farmers Market,Fast Food Restaurant,Field,Fireworks Store,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Golf Course,Golf Driving Range,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indie Movie Theater,Indie Theater,Intersection,Irish Pub,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kitchen Supply Store,Lake,Laundry Service,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Men's Store,Metro Station,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,Nightclub,Noodle House,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Pet Store,Pharmacy,Photography Lab,Pide Place,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Pool,Pool Hall,Poutine Place,Print Shop,Pub,Record Shop,Recording Studio,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Road,Rock Climbing Spot,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Shoe Store,Shop & Service,Shopping Mall,Skate Park,Skating Rink,Ski Area,Ski Chalet,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stationery Store,Steakhouse,Storage Facility,Street Art,Supermarket,Supplement Shop,Taco Place,Tailor Shop,Tanning Salon,Tea Room,Tech Startup,Tennis Court,Theater,Thrift / Vintage Store,Toy / Game Store,Track,Trail,Train Station,Transportation Service,Tree,University,Video Game Store,Video Store,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Zoo
0,Agincourt,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.019608,0.000000,0.039216,0.019608,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.019608,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.019608,0.00,0.019608,0

In [56]:
tor_grouped.head()

,Neighborhood,Accessories Store,Airport,Airport Lounge,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Arcade,Art Gallery,Arts & Crafts Store,Athletics & Sports,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Bike Shop,Bistro,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Camera Store,Candy Store,Castle,Cemetery,Cheese Shop,Chiropractor,Chocolate Shop,Church,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Lab,College Quad,College Rec Center,College Stadium,College Theater,Comedy Club,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Dentist's Office,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Donut Shop,Drugstore,Electronics Store,Event Space,Farm,Farmers Market,Fast Food Restaurant,Field,Fireworks Store,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Golf Course,Golf Driving Range,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indie Movie Theater,Indie Theater,Intersection,Irish Pub,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kitchen Supply Store,Lake,Laundry Service,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Men's Store,Metro Station,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,Nightclub,Noodle House,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Pet Store,Pharmacy,Photography Lab,Pide Place,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Pool,Pool Hall,Poutine Place,Print Shop,Pub,Record Shop,Recording Studio,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Road,Rock Climbing Spot,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Shoe Store,Shop & Service,Shopping Mall,Skate Park,Skating Rink,Ski Area,Ski Chalet,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stationery Store,Steakhouse,Storage Facility,Street Art,Supermarket,Supplement Shop,Taco Place,Tailor Shop,Tanning Salon,Tea Room,Tech Startup,Tennis Court,Theater,Thrift / Vintage Store,Toy / Game Store,Track,Trail,Train Station,Transportation Service,Tree,University,Video Game Store,Video Store,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Zoo
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.019608,0.00000,0.039216,0.019608,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.019608,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.019608,0.0,0.019608,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.019608,0.00000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.019608,0.0,0.0,0.000000,0.019608,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0

#### Print the top 4 venues of each neighborhood

In [52]:
num_top_venues = 4

for hood in tor_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = tor_grouped[tor_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
            venue  freq
0      Restaurant  0.45
1   Shopping Mall  0.08
2     Pizza Place  0.04
3  Sandwich Place  0.04


----Alderwood / Long Branch----
            venue  freq
0        Pharmacy  0.12
1  Discount Store  0.12
2     Pizza Place  0.08
3             Pub  0.04


----Bathurst Manor / Wilson Heights / Downsview North----
         venue  freq
0   Restaurant  0.17
1         Bank  0.07
2  Coffee Shop  0.07
3  Pizza Place  0.07


----Bayview Village----
         venue  freq
0   Restaurant  0.31
1         Bank  0.15
2  Gas Station  0.15
3         Park  0.08


----Bedford Park / Lawrence Manor East----
            venue  freq
0      Restaurant  0.27
1     Coffee Shop  0.07
2  Sandwich Place  0.05
3            Bank  0.05


----Berczy Park----
         venue  freq
0   Restaurant  0.20
1  Coffee Shop  0.12
2        Hotel  0.05
3         Café  0.05


----Birch Cliff / Cliffside West----
               venue  freq
0               Park  0.13
1         Restaurant  0.13


#### Put it into a dataframe

In [53]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Make it display the top 6 venues of each neighborhood

In [55]:
num_top_venues = 6

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = tor_grouped['Neighborhood']

for ind in np.arange(tor_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(tor_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue
0,Agincourt,Restaurant,Shopping Mall,Pizza Place,Sandwich Place,Bakery,Pool Hall
1,Alderwood / Long Branch,Discount Store,Pharmacy,Pizza Place,Gas Station,Convenience Store,Intersection
2,Bathurst Manor / Wilson Heights / Downsview North,Restaurant,Coffee Shop,Pizza Place,Bank,Ice Cream Shop,Bridal Shop
3,Bayview Village,Restaurant,Gas Station,Bank,Café,Grocery Store,Skating Rink
4,Bedford Park / Lawrence Manor East,Restaurant,Coffee Shop,Bank,Fast Food Restaurant,Sandwich Place,Intersection


## Cluster the Neighborhoods into 4 clusters

In [57]:
# set number of clusters
kclusters = 4

tor_grouped_clustering = tor_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(tor_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 0, 0, 2, 2, 1, 0, 1, 1, 0], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 6 venues for each neighborhood.

In [58]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

tor_merged = tor_data

tor_merged = tor_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')


,Postal code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,0.0,Park,Bus Stop,Shopping Mall,Pharmacy,Convenience Store,Restaurant
1,M4A,North York,Victoria Village,43.725882,-79.315572,0.0,Park,Coffee Shop,Men's Store,Golf Course,Sporting Goods Shop,Lounge
2,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636,1.0,Restaurant,Coffee Shop,Park,Diner,Pub,Theater
3,M6A,North York,Lawrence Manor / Lawrence Heights,43.718518,-79.464763,1.0,Restaurant,Furniture / Home Store,Clothing Store,Coffee Shop,Fast Food Restaurant,Fried Chicken Joint
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494,1.0,Restaurant,Coffee Shop,Park,Gastropub,Burger Joint,Yoga Studio


In [69]:
tor_merged = tor_merged.dropna().reset_index().drop(columns = 'index')
tor_merged

,Postal code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,0.0,Park,Bus Stop,Shopping Mall,Pharmacy,Convenience Store,Restaurant
1,M4A,North York,Victoria Village,43.725882,-79.315572,0.0,Park,Coffee Shop,Men's Store,Golf Course,Sporting Goods Shop,Lounge
2,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636,1.0,Restaurant,Coffee Shop,Park,Diner,Pub,Theater
3,M6A,North York,Lawrence Manor / Lawrence Heights,43.718518,-79.464763,1.0,Restaurant,Furniture / Home Store,Clothing Store,Coffee Shop,Fast Food Restaurant,Fried Chicken Joint
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494,1.0,Restaurant,Coffee Shop,Park,Gastropub,Burger Joint,Yoga Studio
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242,0.0,Pharmacy,Skating Rink,Bakery,Café,Park,Shopping Mall
6,M1B,Scarborough,Malvern / Rouge,43.806686,-79.194353,2.0,Restaurant,Fast Food Restaurant,Coffee Shop,Trail,Hobby Shop,Arts & Crafts Store
7,M3B,North York,Don Mills,43.745906,-79.352188,2.0,Restaurant,Coffee Shop,Supermarket,Bank,Gym,Pizza Place
8,M4B,East York,Parkview Hill / Woodbine Gardens,43.706397,-79.309937,0.0,Brewery,Gym / Fitness Center,Coffee Shop,Fast Food Restaurant,Pizza Place,Gastropub
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,1.0,Restaurant,Coffee Shop,Gastropub,Theater,Plaza,Gay Bar


Let's visualize the clusters

In [70]:
#Get geocode of Toronto

latitude = 43.6532
longitude = -79.3832
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6532, -79.3832.


In [73]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(tor_merged['Latitude'], tor_merged['Longitude'], tor_merged['Neighborhood'], tor_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Check the cluster code of our potential neighborhood list

In [75]:
neigh_list

['Fairview / Henry Farm / Oriole',
 'Leaside',
 'Regent Park / Harbourfront',
 'Runnymede / The Junction North']

In [101]:
cluster_dict = {}
for i in neigh_list:
    cluster_dict[i] = tor_merged[tor_merged['Neighborhood'] == i ].values.tolist()[0]

final_list = pd.DataFrame.from_dict(cluster_dict, orient='index', columns = tor_merged.columns)

name_list = final_list['Neighborhood'].tolist()
lat_list = final_list['Latitude'].tolist()
long_list = final_list['Longitude'].tolist()
cluster_list = final_list['Cluster Labels'].tolist()



From the map above we can see that culster 1 locates near to the center of the city. 
Within the three neighborhoods in cluster 1, Leaside is the one with more shops which can attract people for consumption. 
Therefore, Leaside is the best option for opening a new Asian restaurant we propose. 

In [103]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters

folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='blue',
    popup='City Center',
    fill = True,
    fill_color = 'blue',
    fill_opacity = 0.6
).add_to(map_clusters)


# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(lat_list, long_list, name_list, cluster_list):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Results and Conclusion <a name="results"></a>

Our analysis shows that although there is a great number of restaurants in Toronto. There are pockets of low restaurant density fairly far from city center but with an active business environment. With 4 neighborhoods in final potential list, we look into the top venues of each neighborhood along with their cluster. Since cluster 1 consists of neighborhoods which have the most human traffic and active business environment, we filter out neighborhoods that are not in cluster 1. 
Moreover, since neighborhoods close to city center can be over crowded and suffer from the fact that the human traffic may not be sufficient in holidays or weekend, we try to locate the neighborhood that is further from city center. 


*The final decision is to build an Asian restaurant in Leaside.*